In [4]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import glob
import shutil
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.tag as tag
import thor.visualize as visualize

notebook_name = "gridrad_demo.ipynb"
__file__ = "/home/ewan/Documents/THOR/examples/gridrad_demo.ipynb"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2010-01-20T18:00:00"
end = "2010-01-20T19:00:00"

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
gridrad_options = data.gridrad.gridrad_data_options(
    start=start, end=end, converted_options=converted_options
)
era5_pl_options = data.era5.data_options(start=start, end=end)
era5_sl_options = data.era5.data_options(
    start=start, end=end, data_format="single-levels"
)
data_options = option.consolidate_options(
    [gridrad_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, filename="gridrad_era5")

# Create the grid_options dictionary using the first file in the cpol dataset
grid_options = grid.create_options(
    name="geographic", regrid=False, altitude_spacing=None, geographic_spacing=None
)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, filename="gridrad_geographic")

# Create the tag_options dictionary
era5_pl_tag_options = data.era5.tag_options()
era5_sl_tag_options = data.era5.tag_options(dataset="era5_sl")
tag_options = option.consolidate_options([era5_pl_tag_options, era5_sl_tag_options])
tag.save_tag_options(tag_options, filename="era5")

# Create the track_options dictionary
track_options = option.mcs(
    dataset="gridrad",
    tags=["era5_pl", "era5_sl"],
    global_flow_margin=75,
    unique_global_flow=False,
)
option.check_options(track_options)
option.save_track_options(track_options, filename="gridrad_mcs")

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["cell", "anvil", "mcs"]
}
# visualize_options["middle_cloud"] = visualize.option.runtime_options(
#     "cell", save=True, style="presentation", figure_types=["mask"]
# )
visualize.option.save_display_options(visualize_options, filename="runtime_mcs")

2024-08-26 21:10:39,411 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/data_options/gridrad_era5.yaml
2024-08-26 21:10:39,547 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/grid_options/gridrad_geographic.yaml
2024-08-26 21:10:39,550 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/tag_options/era5.yaml
2024-08-26 21:10:39,553 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/track_options/gridrad_mcs.yaml
2024-08-26 21:10:39,564 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/visualize_options/runtime_mcs.yaml


In [18]:
import cProfile
import pstats

In [19]:
profiler = cProfile.Profile()
profiler.enable()

2024-08-26 19:21:07,452 - thor.track - INFO - Processing 2010-01-20T18:00:00.
2024-08-26 19:21:07,456 - thor.data.gridrad - DEBUG - Updating gridrad dataset for 2010-01-20T18:00:00.
2024-08-26 19:21:07,458 - thor.data.gridrad - DEBUG - Converting gridrad data from nexrad_3d_v4_2_20100120T180000Z.nc
2024-08-26 19:21:07,894 - thor.data.gridrad - DEBUG - Filtering GridRad data
2024-08-26 19:21:10,409 - thor.data.gridrad - DEBUG - Removing clutter from the GridRad data
2024-08-26 19:21:11,295 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-08-26 19:21:12,455 - thor.data.gridrad - DEBUG - Removing low level clutter from the GridRad data
2024-08-26 19:21:12,584 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-08-26 19:21:22,349 - thor.track - DEBUG - Processing hierarchy level 0.
2024-08-26 19:21:22,350 - thor.track - DEBUG - Tracking cell.
2024-08-26 19:21:34,477 - thor.match.match - DEBUG - Matching cell objects.
2024-08-26 19:21:34,495 

KeyboardInterrupt: 

In [7]:

output_directory = base_local / "runs/gridrad_demo"
if output_directory.exists():
    shutil.rmtree(output_directory)
times = data.utils.generate_times(data_options["gridrad"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    tag_options,
    visualize_options,
    output_directory=base_local / "runs/gridrad_demo",
)


2024-08-26 21:10:50,464 - thor.track - DEBUG - Beginning simultaneous tracking.
2024-08-26 21:10:50,553 - thor.track - INFO - Processing 2010-01-20T18:00:00.
2024-08-26 21:10:50,555 - thor.data.gridrad - DEBUG - Updating gridrad dataset for 2010-01-20T18:00:00.
2024-08-26 21:10:50,556 - thor.data.gridrad - DEBUG - Converting gridrad data from nexrad_3d_v4_2_20100120T180000Z.nc
2024-08-26 21:10:50,983 - thor.data.gridrad - DEBUG - Filtering GridRad data
2024-08-26 21:10:52,299 - thor.data.gridrad - DEBUG - Removing clutter from the GridRad data
2024-08-26 21:10:52,504 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-08-26 21:10:53,438 - thor.data.gridrad - DEBUG - Removing low level clutter from the GridRad data
2024-08-26 21:10:53,503 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data


In [13]:

profiler.disable()
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats(50)

         103742968 function calls (99721575 primitive calls) in 3099.148 seconds

   Ordered by: cumulative time
   List reduced from 5065 to 50 due to restriction <50>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       27    0.000    0.000 2735.198  101.304 /home/ewan/miniconda3/envs/THOR/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3541(run_code)
       33    0.000    0.000 2734.801   82.873 {built-in method builtins.exec}
        1    0.000    0.000 2734.696 2734.696 /home/ewan/Documents/THOR/thor/track.py:146(simultaneous_track)
       14    0.000    0.000 2666.003  190.429 /home/ewan/Documents/THOR/thor/track.py:226(track_level)
       28    0.001    0.000 2665.976   95.213 /home/ewan/Documents/THOR/thor/track.py:265(track_object)
      367    0.051    0.000 2409.955    6.567 /home/ewan/miniconda3/envs/THOR/lib/python3.10/site-packages/dask/threaded.py:37(get)
      367    1.198    0.003 2409.892    6.566 /home/ewan/miniconda3/envs/THO

In [26]:
visualize.visualize.animate_object("mask", "cell", output_directory)

2024-08-26 20:48:41,747 - thor.visualize.visualize - INFO - Animating mask figures for cell objects on 2010-01-20.
